Credit: https://github.com/tencent-quantum-lab/tensorcircuit/issues/120

# Variational Quantum Eigensolver (VQE) on Molecules

## Overview

VQE is a variational algorithm for calculating the ground state of some given hamiltonian H which we call it $\psi_g$ that satisfied $H \left|\psi_g\right> =E_g\left|\psi_g\right>$. For an arbitrary normalized wavefunction $\psi_f$, the expectation value $\left<\psi_f|H|\psi_f \right>$ is always not lower than the ground state energy unless $\psi_f = \psi_g$ to some phase factor (here we assume there is no degeneracy in ground state). Based on that fact, if we use a parameterized wavefunction $\psi_\theta$, e.g. given by a parameterized quantum circuit (PQC) with parameters $\theta$, we can give an approximation for the ground state enery and wavefunction by minimizing the expectation value of $H$. In practical quantum hardware, this algorithm can be realized in a quantum-neural hybrid paradigm with the gradient calculated using finite difference or paremeter shift in quantum hardware and the optimization using gradient descent method in classical computer. While in a numerical simulation, we can just calculate the gradients using automatic differentiation. 

Calculating the ground state energy for molecules is often important for quantum chemistry tasks since it can be used to find out the atom structure of the molecules. In the simulation of molecules, we do not consider the motion of nuclei which means we fix the nuclear coordinates of its constituent atoms. We only consider the electrons in the molecules since the nuclei are way heavier than the electrons and thus the energy carried by phonons is negligible or can be reconsidered using Born-Oppenheimer approximation. Strictly speaking, the eletrons lives in continuous space, thus the Hilbert space is of infinite dimensions. To conduct a practical calculation, we only preserve some important single-particle basis, e.g. the low energy atomic orbitals. In the second quantization formalism, we can represent these atomic orbitals as $c_i^\dagger|0>$. By considering the interactions of nuclei and electrons as background and also the electron-electron interaction, a molecules hamiltonian can in generally be represented as $H = \sum_{i, j} h_{i,j} c_i^\dagger c_j + \sum_{i, j, k, l} \alpha_{i, j, k, l} c_i^\dagger c_j^\dagger c_k c_l$. Notice that the spin index is also absorbed into the orbital index. There are many softwares that can give these parameters in H such as `pyscf` which we will use later in this tutorial. Now we have a fermionic description for moleculars. By using a mapping from fermionic operators to spin operators such as Jordan-Wigner transformation or Bravyi-Kitaev transformation, we can map the fermionic hamiltonian to a spin hamiltonian which is more compatible with quantum computer. For a spin hamiltonian, we can easily use a PQC to construct a trail wavefunction and conduct the VQE algorithm. In the following part of this tutorial, we will demonstrate a complete example of how to use TensorCircuit to simulate VQE algorithm on Molecules.

## Setup

We should first ``pip install openfermion openfermionpyscf`` to generate fermionic and qubit Hamiltonian of CO2 molecule based on quantum chemistry computation provided by openfermion and pyscf.

In [1]:
!pip3 install --upgrade openfermion openfermionpyscf tensorflow tensorcircuit

Requirement already up-to-date: openfermion in /home/moe/.local/lib/python3.8/site-packages (1.5.1)
Requirement already up-to-date: openfermionpyscf in /home/moe/.local/lib/python3.8/site-packages (0.5)
Requirement already up-to-date: tensorflow in /home/moe/.local/lib/python3.8/site-packages (2.11.0)
Requirement already up-to-date: tensorcircuit in /home/moe/.local/lib/python3.8/site-packages (0.7.0)


In [2]:
import numpy as np
from openfermion.chem import MolecularData
from openfermion.transforms import (
    get_fermion_operator,
    jordan_wigner,
    binary_code_transform,
    checksum_code,
    reorder,
)
from openfermion.chem import geometry_from_pubchem
from openfermion.utils import up_then_down
from openfermion.linalg import LinearQubitOperator
from openfermionpyscf import run_pyscf
import tensorflow as tf

import tensorcircuit as tc

K = tc.set_backend("tensorflow")

2023-02-04 14:42:20.605315: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-04 14:42:20.743267: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-02-04 14:42:20.743296: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-04 14:42:21.469284: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared ob

## Generate Hamiltonian

* Get molecule energy info and molecule orbitals

In [3]:
multiplicity = 1
basis = "sto-3g"
# 15 spin orbitals for CO2
geometry = geometry_from_pubchem("co2")
description = "co2"
molecule = MolecularData(geometry, basis, multiplicity, description=description)
molecule = run_pyscf(molecule, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)
print(molecule.fci_energy, molecule.ccsd_energy, molecule.hf_energy)

-185.2925660467095 -185.26927761956804 -185.068031076591


* Find the number of spin orbitals for H2O and CO2

In [4]:
molecule_h2o = MolecularData(geometry_from_pubchem("h2o"), basis, multiplicity, description="H2O")
molecule_h2o = run_pyscf(molecule_h2o, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)
print("Number of spin orbitals for H2O:", molecule_h2o.n_orbitals)

molecule_co2 = MolecularData(geometry_from_pubchem("co2"), basis, multiplicity, description="CO2")
molecule_co2 = run_pyscf(molecule_co2, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)
print("Number of spin orbitals for CO2:", molecule_co2.n_orbitals)

Number of spin orbitals for H2O: 7
Number of spin orbitals for CO2: 15


* Some explanation on the discrepency between theory and code output:

The number of spin orbitals for a molecule can be calculated based on the number of electrons in the molecule. Each electron will occupy one spin orbital, and there are two spin orbitals per occupied orbital: one with a spin-up electron and one with a spin-down electron. The total number of spin orbitals is then equal to 2 times the number of electrons.

For H2O, there are 8 electrons in the molecule, so the number of spin orbitals is 2 * 8 = 16.

For CO2, there are 16 electrons in the molecule, so the number of spin orbitals is 2 * 16 = 32.


The number of spin orbitals can be found from the molecule object returned by the run_pyscf function. In the case of H2O, it returns 7 spin orbitals and in the case of CO2, it returns 15 spin orbitals.

This is because the number of spin orbitals is proportional to the number of electrons in the molecule and the type of basis set used to describe the molecule's electronic structure. The sto-3g basis set used in these examples is a relatively small basis set, which results in a smaller number of spin orbitals.

* Get Fermionic Hamiltonian

In [5]:
mh = molecule.get_molecular_hamiltonian()

In [6]:
fh = get_fermion_operator(mh)

The one-particle operator acts on a single particle and corresponds to one-body terms in the Hamiltonian, while the two-particle operator acts on two particles and corresponds to two-body terms in the Hamiltonian. The terms in the Hamiltonian can be represented by either fermionic or qubit operators, and the coefficients of these terms give the energy contributions from each term to the total energy of the system.

The notation `(C_0^\dagger C_1^\dagger)(C_0 C_1)` represents a two-particle operator. It's known as the Slater determinant. The terms `C_0^\dagger C_1^\dagger` and `C_0 C_1` represent the creation and annihilation of two fermions, respectively. The two-particle operator as a whole represents the exchange of two fermions, one created and one annihilated.

The notation `(C_0^\dagger C_1^\dagger)(C_0 C_1)` represents a two-particle operator that acts on two fermions in two different states or orbitals. The two-particle operator as a whole can be thought of as a representation of the exchange of two fermions, one created in state 1 and one annihilated in state 0. It represents the interaction between two fermions, as opposed to a single fermion.

In quantum mechanics, the Slater determinant is a way to represent the wave function of a many-body system made up of fermions. It is antisymmetric, meaning that exchanging two fermions in the wave function leads to a sign change. This is known as the Pauli exclusion principle, which states that no two fermions can occupy the same quantum state simultaneously.

In [7]:
print(fh.terms[((0, 1), (0, 0))])  # coefficient of (C_0^\dagger C_1^\dagger)(C_0 C_1)

-36.019328404244526


The notation `C_0^\dagger C_0` represents the creation operator `C_0^\dagger` acting on the state with 0 electrons, which results in a state with one electron. Here, `C_0^\dagger` creates a particle in the "0" orbital, and `C_0` annihilates a particle in the "0" orbital. The product of these two operators gives the number of particles in the "0" orbital.

The `None` output below indicates that there is no term in the FermionOperator corresponding to the one-particle operator `C_0^\dagger C_0`. This is likely because the Hamiltonian of the CO2 molecule is more complex than a one-particle operator and requires the use of two-particle terms to describe the interactions between the particles.

In [8]:
print(fh.terms.get(((0, 1),)))  # coefficient of C_0^\dagger C_0
#print(fh.terms[((0, 1),)])  # this returns KeyError, so use .get() instead

None


This following code loops over the terms in `fh.terms` and checks if the term has two elements, which represent the creation and annihilation operators. It then checks if the first character of both elements is "c", which means they are creation or annihilation operators, and prints the corresponding index and coefficient of the term.

The reason for only considering terms with one creation and one annihilation operator is because such terms correspond to a single-particle operator, which can be represented by a matrix. In many quantum chemistry problems, the Hamiltonian is expressed as a sum of these single-particle operators. The single-particle operators can be expressed as the outer product of two creation/annihilation operators, hence the constraint of only considering terms with one creation and one annihilation operator.

In [9]:
one_body_terms = fh.terms
for term, coefficient in one_body_terms.items():
    if len(term) == 2:
        # Only consider terms with one creation and one annihilation operator
        i, j = term
        print("Term: c^\dagger_{} c_{} Coefficient: {}".format(i[0], j[0], coefficient))


Term: c^\dagger_0 c_0 Coefficient: -36.019328404244504
Term: c^\dagger_0 c_8 Coefficient: -0.5334287454767641
Term: c^\dagger_0 c_16 Coefficient: -0.314813566605117
Term: c^\dagger_0 c_28 Coefficient: 0.24049575887623442
Term: c^\dagger_1 c_1 Coefficient: -36.019328404244504
Term: c^\dagger_1 c_9 Coefficient: -0.5334287454767641
Term: c^\dagger_1 c_17 Coefficient: -0.314813566605117
Term: c^\dagger_1 c_29 Coefficient: 0.24049575887623442
Term: c^\dagger_2 c_2 Coefficient: -36.0197626223573
Term: c^\dagger_2 c_4 Coefficient: 0.0002660337987218859
Term: c^\dagger_2 c_6 Coefficient: 0.48949009290870976
Term: c^\dagger_2 c_10 Coefficient: -0.3955819317157017
Term: c^\dagger_2 c_26 Coefficient: -0.18321310762007603
Term: c^\dagger_3 c_3 Coefficient: -36.0197626223573
Term: c^\dagger_3 c_5 Coefficient: 0.0002660337987218859
Term: c^\dagger_3 c_7 Coefficient: 0.48949009290870976
Term: c^\dagger_3 c_11 Coefficient: -0.3955819317157017
Term: c^\dagger_3 c_27 Coefficient: -0.18321310762007603
Te

* Transform into qubit Hamiltonian

In [10]:
# The normal transformation such as JW or BK requires 14 qubits for CO2's 30 orbitals

a = jordan_wigner(fh)
LinearQubitOperator(a).n_qubits

30

We can use binary code to save two further qubits, as the number of spin up and spin down filling is both 13 (13/odd electrons in 15 orbitals)

In [11]:
b = binary_code_transform(reorder(fh, up_then_down), 2 * checksum_code(molecule_co2.n_orbitals, 1))
# molecule_co2.n_orbitals = 15
# 15 is 15 spin polarized orbitals, and 1 is for odd occupation
LinearQubitOperator(b).n_qubits

28

In [12]:
print(b.terms[((0, "Z"),)])  # coefficient of Z_0 Pauli-string

12.934425376117849


* Transform the qubit Hamiltonian in openfermion to the format in TensorCircuit

In [13]:
lsb, wb = tc.templates.chems.get_ps(b, 2*(molecule_co2.n_orbitals-1))
lsa, wa = tc.templates.chems.get_ps(a, 2*molecule_co2.n_orbitals)

* Inspect Hamiltonian in matrix form

In [14]:
ma = tc.quantum.PauliStringSum2COO_numpy(lsa, wa)

2023-02-04 14:43:53.321861: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-02-04 14:43:53.358407: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-04 14:43:53.358463: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (t1v-n-ff0a25ed-w-0): /proc/driver/nvidia/version does not exist
2023-02-04 14:43:53.466122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-04 14:43:55.406

: 

: 

In [ ]:
mb = tc.quantum.PauliStringSum2COO_numpy(lsb, wb)

In [ ]:
mad, mbd = ma.todense(), mb.todense()

The corresponding Hartree Fock product state in these two types of Hamiltonian

In [ ]:
bin(np.argmin(np.diag(mad)))

In [ ]:
bin(np.argmin(np.diag(mbd)))

## VQE Setup

We can in principle evaluate each Pauli string of the Hamiltonian as an expectation measurement, but it costs lots of simulation time, instead we fuse them as a Hamiltonian matrix as shown above to run the VQE.

* Using dense matrix expectation

In [ ]:
n = 12
depth = 4
mbd_tf = tc.array_to_tensor(mbd)


def vqe(param):
    c = tc.Circuit(n)
    for i in [0, 1, 2, 3, 4, 6, 7, 8, 9, 10]:
        c.X(i)
    for j in range(depth):
        for i in range(n - 1):
            c.exp1(i, i + 1, unitary=tc.gates._xx_matrix, theta=param[j, i, 0])
        for i in range(n):
            c.rx(i, theta=param[j, i, 1])
        for i in range(n):
            c.ry(i, theta=param[j, i, 2])
        for i in range(n):
            c.rx(i, theta=param[j, i, 3])
    return tc.templates.measurements.operator_expectation(c, mbd_tf)

In [ ]:
vags = tc.backend.jit(tc.backend.value_and_grad(vqe))
lr = tf.keras.optimizers.schedules.ExponentialDecay(
    decay_rate=0.5, decay_steps=300, initial_learning_rate=0.5e-2
)
opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))

param = tc.backend.implicit_randn(shape=[depth, n, 4], stddev=0.02, dtype="float32")
for i in range(600):
    e, g = vags(param)
    param = opt.update(g, param)
    if i % 100 == 0:
        print(e)

* Using sparse matrix expectation

We can also use the sparse Hamiltonian matrix for circuit expectation evaluation, the only difference is to replace ``mbd_tf`` with ``mb_tf``

In [ ]:
mb_tf = tc.backend.coo_sparse_matrix(
    np.transpose(np.stack([mb.row, mb.col])), mb.data, shape=(2**n, 2**n)
)

A micro-benchmark between sparse matrix evaluation and dense matrix evaluation for expectation in terms of time, sparse always wins in terms of space, of course.

In [ ]:
def dense_expt(param):
    c = tc.Circuit(n)
    for i in range(n):
        c.H(i)
        c.rx(i, theta=param[i])
    return tc.templates.measurements.operator_expectation(c, mbd_tf)


def sparse_expt(param):
    c = tc.Circuit(n)
    for i in range(n):
        c.H(i)
        c.rx(i, theta=param[i])
    return tc.templates.measurements.operator_expectation(c, mb_tf)

In [ ]:
dense_vag = tc.backend.jit(tc.backend.value_and_grad(dense_expt))
sparse_vag = tc.backend.jit(tc.backend.value_and_grad(sparse_expt))

v0, g0 = dense_vag(tc.backend.ones([n]))
v1, g1 = sparse_vag(tc.backend.ones([n]))

# consistency check

np.testing.assert_allclose(v0, v1, atol=1e-5)
np.testing.assert_allclose(g0, g1, atol=1e-5)

In [ ]:
%timeit dense_vag(tc.backend.ones([n]))

In [ ]:
%timeit sparse_vag(tc.backend.ones([n]))

Therefore, sparse matrix evaluation also saves time apart from space, which is always recommended.